In [ ]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os


In [ ]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [ ]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".

*******************************************************************************************
*******************************************************************************************
STARTING WITH FRAX

In [261]:
#https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet/
#frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')
frax = sg.load_api('https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet')

In [262]:
fraxCollateralQuery = frax.Query.pairPerDays(
     orderBy='id'
    ,orderDirection='desc'
    ,first=1000
    #,skipValue = skipValue
    #,where = [frax.Query.fraxCollateral.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
    ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
             #,'timestamp_gte': maxBorrowFraxTimestamp
             }

)

fraxCollateral= sg.query_df([
    fraxCollateralQuery.id
    ,fraxCollateralQuery.timestamp
    ,fraxCollateralQuery.pair.address
    ,fraxCollateralQuery.pair.symbol
    ,fraxCollateralQuery.interestPerSecond #gotta convert that to APY
    ,fraxCollateralQuery.totalBorrowValue #total borrowed
    ,fraxCollateralQuery.utilization #these two give you asset available
    ,fraxCollateralQuery.pair.siloCollateral
    ,fraxCollateralQuery.totalCollateralValue
    ,fraxCollateralQuery.totalAssetValue

    ]
)

fraxCollateral.head(1000)

,pairPerDays_id,pairPerDays_timestamp,pairPerDays_pair_address,pairPerDays_pair_symbol,pairPerDays_interestPerSecond,pairPerDays_totalBorrowValue,pairPerDays_utilization,pairPerDays_pair_collateral_id,pairPerDays_pair_collateral_address,pairPerDays_pair_collateral_decimals,pairPerDays_pair_collateral_name,pairPerDays_pair_collateral_symbol,pairPerDays_pair_collateral_totalSupply,pairPerDays_pair_collateral_pairAssetCount,pairPerDays_pair_collateral_pairCollateralCount,pairPerDays_totalCollateralValue,pairPerDays_totalAssetValue
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,1676494787,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,2984304040,400116192946270617921986200000000,85197,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,8403423075666293637915414368053297717111857,469631799103375165432615200000000
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19403,1676494787,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,2984304040,400116192946270617921986200000000,85197,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,8403423075666293637915414368053297717111857,469631799103375165432615200000000
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19402,1676418959,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,3000793398,401030577997982608018279400000000,85776,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,8343388959777369440672030013566457194160780,467531536062129324318307600000000
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19401,1676332535,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,2655789209,393433850641671840122635700000000,75450,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,8074656854825809483692286131603213781297678,521447875408495707724551200000000
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19400,1676246147,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,2625172870,388538769723592668721207100000000,74524,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,7725593430519191381171962996564285586737875,521359330105363815236791200000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19330,1670198135,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,887806114,164224614112999780177884700000000,54736,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,4082343490859501384694338597284878133749520,300026579974906776955355200000000
75,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19329,1670111891,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,797470641,135868742062949858845517400000000,45283,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,3355760552996256498800322075345406432876996,300040741776283087084536000000000
76,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19328,1670025359,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,819794982,128753870536361513515734000000000,42913,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,3060333740445465616397882086195529805596420,300031641781023665749242900000000
77,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19327,1669938935,0x66bf36dba79d4606039f04b32946a260bcd3ff52,fFRAX(gOHM)-11,158550393,18212764207485974382800000000,18,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,18,Governance OHM,gOHM,71757394079134961624973,0,1,2560

In [268]:
siloCollateral = sum(fraxCollateral['pairPerDays_totalCollateralValue'])
print('There are',round(siloCollateral,2),  'gOHM available to borrow')

There are 551836826645851149701865767669449099800090842 gOHM available to borrow


*******************************************************************************************
*******************************************************************************************
SILO

In [ ]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

#OHM pair can be found at https://app.silo.finance/silo/0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db
#GOHM pair at https://app.silo.finance/silo/0xc413dd03555f3eb29d834b482d386b2999dc2eb0 THIS THE ONE YOU WANT

In [242]:
oldDepositsDb = pd.DataFrame()
siloFileName = file+'/siloFinance_deposits.csv'
try:
    oldDepositsDb = pd.read_csv (siloFileName)
except:
    oldDepositsDb = pd.DataFrame()
oldDepositsDb.head(10)

""


In [243]:
maxSiloDepositTimestamp = 0
try:
    maxSiloDepositTimestamp = (max(0,oldDepositsDb['transaction_timestamp'].max()))-5
except:
        maxSiloDepositTimestamp = 0
maxSiloDepositTimestamp = int(maxSiloDepositTimestamp)
maxSiloDepositTimestamp

0

In [244]:
#this tracks the ENTIRE list of events for depositing gOHM for lending
depositTicker = 0
depositslist = pd.DataFrame()
depositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where=
        { 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'  #gOHM market
                ,'timestamp_gt': maxSiloDepositTimestamp
                ,'asset': '0x0ab87046fbb341d058f17cbc4c1133f25a20a52f'
#            [ silo.Query.deposits.market == '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
#             ,silo.Query.deposits.timestamp > maxSiloDepositTimestamp
#             ,silo.Query.deposits.asset == '0x0ab87046fbb341d058f17cbc4c1133f25a20a52f']
                }


    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account.id
        ,deposits.market.id
        ,deposits.market.protocol.id
        ,deposits.market.canUseAsCollateral
        ,deposits.market.rates.rate
        ,deposits.market.rates.duration
        ,deposits.market.rates.side
        ,deposits.market.rates.type
        ,deposits.market.rates.token.id
        ,deposits.asset.id
        ,deposits.asset.name
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.amount
        ,deposits.amountUSD
        ,deposits.position.timestampClosed
    ])

    #depositslist.sort_values(by=['deposits_timestamp'])

    depositsDb=pd.concat([depositsDb, depositslist])
    recordTimestamp1 = depositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    depositsDbLength = len(depositsDb)
    depositListLength = len(depositslist)
    recordID = depositsDb.iat[depositTicker,0]
    deposit = depositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", depositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", depositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1
    if daysAgo>daysLimit: exit =True
    if depositListLength<1000: exit =True

#depositsDb.sort_values(by=['deposits_timestamp'])

depositsDb.columns = depositsDb.columns.str.replace("deposits", "transaction")
depositsDb['event']='deposit'
depositsDb.drop_duplicates
print("Done. Total Records collected: ", depositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  186  - latest recordID:  0xe5163797f0922e4bded89126a213296a8c3d270a39e2fd0b29379195ff74e9cf   0xe5163797f0922e4bded89126a213296a8c3d270a39e2fd0b29379195ff74e9cf-254  - deposits DB length:  186  - latest record from:  2023-02-14 06:11:23 1 days ago
Done. Total Records collected:  186


In [245]:
#I swear I will figure out synthetic fields, but until then...
depositsDbTimed = db.query("select distinct"
               "            transaction_id "
                           " ,transaction_blockNumber "
                           ",transaction_account_id "
                           ",transaction_market_id "
                           ", transaction_market_canUseAsCollateral "
                           ", transaction_asset_symbol "
                           ",transaction_asset_decimals"
                           ",transaction_asset_lastPriceUSD "
                           ",transaction_amount "
                           ",transaction_amount*10^(-transaction_asset_decimals) naturalAmount"
                           " ,transaction_hash "
                    "        ,to_timestamp(transaction_timestamp::int) dateTime "
                    "        ,date_trunc('day',to_timestamp(transaction_timestamp::int)) calDate "
               "From depositsDb "
               "where transaction_position_timestampClosed::bigint>0"
               #" limit 100 "
               ).df()
depositsDbTimed.head(1000)

,transaction_id,transaction_blockNumber,transaction_account_id,transaction_market_id,transaction_market_canUseAsCollateral,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,naturalAmount,transaction_hash,dateTime,calDate
0,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,16589368,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.190000e+20,119.000000,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,2023-02-09 06:19:47,2023-02-09
1,0x6921d4de2e5fd797205f66357da959b3e4922760d131...,16589333,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.000000e+18,1.000000,0x6921d4de2e5fd797205f66357da959b3e4922760d131...,2023-02-09 06:12:47,2023-02-09
2,0x35d5e6734b990e7e7e6493344db464f86e9fe2318e7b...,16579812,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.500000e+19,25.000000,0x35d5e6734b990e7e7e6493344db464f86e9fe2318e7b...,2023-02-07 22:14:47,2023-02-07
3,0x7811e7757339551966af37751af4b56c16a3e141d682...,16366137,0x25fdda95197f804316f5ba8dbb66ac173ee19981,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.252766e+18,2.252766,0x7811e7757339551966af37751af4b56c16a3e141d682...,2023-01-09 02:05:11,2023-01-09
4,0x7c831909c84d8b801b25a2a939ec2a6d08e7155fac91...,16176119,0x791d1ec51d931186c1d4b80e753b7155dd98f741,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+16,0.020000,0x7c831909c84d8b801b25a2a939ec2a6d08e7155fac91...,2022-12-13 13:36:47,2022-12-13
5,0x6c4f8bc5f49d55b317c6a9ed6d523a662df0c9b330a7...,16170980,0xee1520f94f304e8d551cbf310fe214212e3ca34a,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+19,20.000000,0x6c4f8bc5f49d55b317c6a9ed6d523a662df0c9b330a7...,2022-12-12 20:22:47,2022-12-12
6,0x53d9eed3930a371dbd133a9260a3ad8d5e7d3c4e446a...,16036013,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.514000e+16,0.025140,0x53d9eed3930a371dbd133a9260a3ad8d5e7d3c4e446a...,2022-11-23 23:39:11,2022-11-23
7,0x816d41605087393280db2cf4083d1126a9b62542f79c...,16036005,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.431000e+17,0.143100,0x816d41605087393280db2cf4083d1126a9b62542f79c...,2022-11-23 23:37:35,2022-11-23


In [246]:
depositsDbTimed=pd.concat([depositsDbTimed,oldDepositsDb])
depositsDbTimed.drop_duplicates(keep='first' )
depositsDbTimed.head(100)

,transaction_id,transaction_blockNumber,transaction_account_id,transaction_market_id,transaction_market_canUseAsCollateral,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,naturalAmount,transaction_hash,dateTime,calDate
0,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,16589368,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.190000e+20,119.000000,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,2023-02-09 06:19:47,2023-02-09
1,0x6921d4de2e5fd797205f66357da959b3e4922760d131...,16589333,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.000000e+18,1.000000,0x6921d4de2e5fd797205f66357da959b3e4922760d131...,2023-02-09 06:12:47,2023-02-09
2,0x35d5e6734b990e7e7e6493344db464f86e9fe2318e7b...,16579812,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.500000e+19,25.000000,0x35d5e6734b990e7e7e6493344db464f86e9fe2318e7b...,2023-02-07 22:14:47,2023-02-07
3,0x7811e7757339551966af37751af4b56c16a3e141d682...,16366137,0x25fdda95197f804316f5ba8dbb66ac173ee19981,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.252766e+18,2.252766,0x7811e7757339551966af37751af4b56c16a3e141d682...,2023-01-09 02:05:11,2023-01-09
4,0x7c831909c84d8b801b25a2a939ec2a6d08e7155fac91...,16176119,0x791d1ec51d931186c1d4b80e753b7155dd98f741,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+16,0.020000,0x7c831909c84d8b801b25a2a939ec2a6d08e7155fac91...,2022-12-13 13:36:47,2022-12-13
5,0x6c4f8bc5f49d55b317c6a9ed6d523a662df0c9b330a7...,16170980,0xee1520f94f304e8d551cbf310fe214212e3ca34a,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+19,20.000000,0x6c4f8bc5f49d55b317c6a9ed6d523a662df0c9b330a7...,2022-12-12 20:22:47,2022-12-12
6,0x53d9eed3930a371dbd133a9260a3ad8d5e7d3c4e446a...,16036013,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.514000e+16,0.025140,0x53d9eed3930a371dbd133a9260a3ad8d5e7d3c4e446a...,2022-11-23 23:39:11,2022-11-23
7,0x816d41605087393280db2cf4083d1126a9b62542f79c...,16036005,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.431000e+17,0.143100,0x816d41605087393280db2cf4083d1126a9b62542f79c...,2022-11-23 23:37:35,2022-11-23


In [247]:
path =file+'/siloFinance_deposits.csv'
depositsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_deposits_piped.csv'
depositsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', len(depositsDbTimed),' records')

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/Lending/results/Collateral/siloFinance_deposits_piped.csv file saved, 8  records


In [248]:
collateral = sum(depositsDbTimed['naturalAmount'])
print('There are',round(collateral,2),  'gOHM available to borrow')

There are 167.44 gOHM available to borrow


In [251]:
dailySiloCollateral = depositsDbTimed.groupby('calDate')['naturalAmount'].sum()
dailySiloCollateral.head(100)

calDate
2022-11-23      0.168240
2022-12-12     20.000000
2022-12-13      0.020000
2023-01-09      2.252766
2023-02-07     25.000000
2023-02-09    120.000000
Name: naturalAmount, dtype: float64

In [252]:
oldWithdrawalsDb = pd.DataFrame()
siloFileName = file+'/siloFinance_Withdrawals.csv'
try:
    oldWithdrawalsDb = pd.read_csv (siloFileName)
except:
    oldWithdrawalsDb = pd.DataFrame()
oldWithdrawalsDb.head(10)

,transaction_id,transaction_blockNumber,transaction_account_id,transaction_market_id,transaction_market_canUseAsCollateral,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,naturalAmount,transaction_hash,dateTime,calDate
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.500004e+19,25.000039,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,2023-02-15 15:27:47,2023-02-15
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,6.439304e+19,64.393035,0x85b689883076aa92af0a2356f8e4084610578bb48590...,2023-02-15 14:49:59,2023-02-15
2,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,3.600000e+19,36.000000,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,2023-02-14 16:23:59,2023-02-14
3,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,16589407,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.960712e+19,19.607121,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,2023-02-09 06:27:35,2023-02-09
4,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,16574012,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.947982e+17,0.194798,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,2023-02-07 02:45:47,2023-02-07
5,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,16483146,0xee1520f94f304e8d551cbf310fe214212e3ca34a,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+19,20.000000,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,2023-01-25 10:09:35,2023-01-25
6,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,16366146,0x25fdda95197f804316f5ba8dbb66ac173ee19981,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.252766e+18,2.252766,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,2023-01-09 02:06:59,2023-01-09
7,0x3cf3a5658c3ef1a247ec241370a3abfeb673251eafec...,16176122,0x791d1ec51d931186c1d4b80e753b7155dd98f741,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+16,0.020000,0x3cf3a5658c3ef1a247ec241370a3abfeb673251eafec...,2022-12-13 13:37:23,2022-12-13


In [253]:
maxSiloWithdrawalTimestamp = 0
try:
    maxSiloWithdrawalTimestamp = (max(0,oldWithdrawalsDb['transaction_timestamp'].max()))-5
except:
        maxSiloWithdrawalTimestamp = 0
maxSiloWithdrawalTimestamp = int(maxSiloWithdrawalTimestamp)
maxSiloWithdrawalTimestamp

0

In [254]:
#this tracks the ENTIRE list of events for withdrawaling gOHM for lending
withdrawalTicker = 0
withdrawalslist = pd.DataFrame()
withdrawalsDb = pd.DataFrame()
withdrawalListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (withdrawalTicker)*(1000)
    withdrawals = silo.Query.withdraws(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where=
        { 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'  #gOHM market
                ,'timestamp_gt': maxSiloWithdrawalTimestamp
                ,'asset': '0x0ab87046fbb341d058f17cbc4c1133f25a20a52f'
#            [ silo.Query.withdrawals.market == '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
#             ,silo.Query.withdrawals.timestamp > maxSilowithdrawalTimestamp
#             ,silo.Query.withdrawals.asset == '0x0ab87046fbb341d058f17cbc4c1133f25a20a52f']
                }


    )

    withdrawalslist = sg.query_df([
        withdrawals.id
        ,withdrawals.hash
        ,withdrawals.blockNumber
        ,withdrawals.timestamp
        ,withdrawals.account.id
        ,withdrawals.market.id
        ,withdrawals.market.protocol.id
        ,withdrawals.market.canUseAsCollateral
        ,withdrawals.market.rates.rate
        ,withdrawals.market.rates.duration
        ,withdrawals.market.rates.side
        ,withdrawals.market.rates.type
        ,withdrawals.market.rates.token.id
        ,withdrawals.asset.id
        ,withdrawals.asset.name
        ,withdrawals.asset.symbol
        ,withdrawals.asset.decimals
        ,withdrawals.asset.lastPriceUSD
        ,withdrawals.amount
        ,withdrawals.amountUSD
        ,withdrawals.position.timestampClosed
    ])

    #withdrawalslist.sort_values(by=['withdrawals_timestamp'])

    withdrawalsDb=pd.concat([withdrawalsDb, withdrawalslist])
    recordTimestamp1 = withdrawalsDb.iat[withdrawalTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    withdrawalsDbLength = len(withdrawalsDb)
    withdrawalListLength = len(withdrawalslist)
    recordID = withdrawalsDb.iat[withdrawalTicker,0]
    withdrawal = withdrawalsDb.iat[withdrawalTicker,1]
    print("iterations: ", withdrawalTicker, "Lines skipped: ",skipValue, "records collected: ", withdrawalListLength, " - latest recordID: ",withdrawal, " ", recordID , " - withdrawals DB length: ", withdrawalsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", withdrawalTicker, "records collected: ", withdrawalListLength, " - userdb length: ", withdrawalsDbLength)
    withdrawalTicker = withdrawalTicker+1
    if daysAgo>daysLimit: exit =True
    if withdrawalListLength<1000: exit =True

#withdrawalsDb.sort_values(by=['withdrawals_timestamp'])

withdrawalsDb.columns = withdrawalsDb.columns.str.replace("withdraws", "transaction")
withdrawalsDb['event']='withdraw'
withdrawalsDb.drop_duplicates
print("Done. Total Records collected: ", withdrawalsDbLength)

iterations:  0 Lines skipped:  0 records collected:  72  - latest recordID:  0xf4707edc29622f90030af624d1918ff5ae613cef90d8f6934404ede182d39aba   0xf4707edc29622f90030af624d1918ff5ae613cef90d8f6934404ede182d39aba-260  - withdrawals DB length:  72  - latest record from:  2023-02-15 07:27:47 0 days ago
Done. Total Records collected:  72


In [255]:
withdrawalsDb.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_canUseAsCollateral,transaction_market_rates_rate,transaction_market_rates_duration,...,transaction_market_rates_token_id,transaction_asset_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,transaction_position_timestampClosed,event
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,0.000000,None,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2855.939479,25000038820033231117,71148.084004,1.676475e+09,withdraw
1,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,0.000000,None,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2855.939479,25000038820033231117,71148.084004,1.676475e+09,withdraw
2,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,1.170737,None,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2855.939479,25000038820033231117,71148.084004,1.676475e+09,withdraw
3,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,0.000000,None,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2855.939479,25000038820033231117,71148.084004,1.676475e+09,withdraw
4,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,0.000000,None,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2855.939479,25000038820033231117,71148.084004,1.676475e+09,withdraw
5,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,0.274125,None,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2855.939479,25000038820033231117,71148.084004,1.676475e+09,withdraw
6,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,0.000000,None,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2855.939479,64393035133917233507,183257.358356,1.676473e+09,withdraw
7,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,True,0.000000,None,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0ab87046fbb341d05

In [256]:
#I swear I will figure out synthetic fields, but until then...
withdrawalsDbTimed = db.query("select distinct"
               "            transaction_id "
                           " ,transaction_blockNumber "
                           ",transaction_account_id "
                           ",transaction_market_id "
                           ", transaction_market_canUseAsCollateral "
                           ", transaction_asset_symbol "
                           ",transaction_asset_decimals"
                           ",transaction_asset_lastPriceUSD "
                           ",transaction_amount "
                           ",transaction_amount*10^(-transaction_asset_decimals) naturalAmount"
                           " ,transaction_hash "
                    "        ,to_timestamp(transaction_timestamp::int) dateTime "
                    "        ,date_trunc('day',to_timestamp(transaction_timestamp::int)) calDate "
               "From withdrawalsDb "
               "where transaction_position_timestampClosed::bigint>0"
               #" limit 100 "
               ).df()
withdrawalsDbTimed.head(1000)

,transaction_id,transaction_blockNumber,transaction_account_id,transaction_market_id,transaction_market_canUseAsCollateral,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,naturalAmount,transaction_hash,dateTime,calDate
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.500004e+19,25.000039,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,2023-02-15 15:27:47,2023-02-15
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,6.439304e+19,64.393035,0x85b689883076aa92af0a2356f8e4084610578bb48590...,2023-02-15 14:49:59,2023-02-15
2,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,3.600000e+19,36.000000,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,2023-02-14 16:23:59,2023-02-14
3,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,16589407,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.960712e+19,19.607121,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,2023-02-09 06:27:35,2023-02-09
4,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,16574012,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.947982e+17,0.194798,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,2023-02-07 02:45:47,2023-02-07
5,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,16483146,0xee1520f94f304e8d551cbf310fe214212e3ca34a,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+19,20.000000,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,2023-01-25 10:09:35,2023-01-25
6,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,16366146,0x25fdda95197f804316f5ba8dbb66ac173ee19981,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.252766e+18,2.252766,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,2023-01-09 02:06:59,2023-01-09
7,0x3cf3a5658c3ef1a247ec241370a3abfeb673251eafec...,16176122,0x791d1ec51d931186c1d4b80e753b7155dd98f741,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+16,0.020000,0x3cf3a5658c3ef1a247ec241370a3abfeb673251eafec...,2022-12-13 13:37:23,2022-12-13


In [257]:
withdrawalsDbTimed=pd.concat([withdrawalsDbTimed,oldWithdrawalsDb])
withdrawalsDbTimed.drop_duplicates(keep='first' )
withdrawalsDbTimed.head(100)

,transaction_id,transaction_blockNumber,transaction_account_id,transaction_market_id,transaction_market_canUseAsCollateral,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,naturalAmount,transaction_hash,dateTime,calDate
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.500004e+19,25.000039,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,2023-02-15 15:27:47,2023-02-15 00:00:00
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,6.439304e+19,64.393035,0x85b689883076aa92af0a2356f8e4084610578bb48590...,2023-02-15 14:49:59,2023-02-15 00:00:00
2,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,3.600000e+19,36.000000,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,2023-02-14 16:23:59,2023-02-14 00:00:00
3,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,16589407,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.960712e+19,19.607121,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,2023-02-09 06:27:35,2023-02-09 00:00:00
4,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,16574012,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,1.947982e+17,0.194798,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,2023-02-07 02:45:47,2023-02-07 00:00:00
5,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,16483146,0xee1520f94f304e8d551cbf310fe214212e3ca34a,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+19,20.000000,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,2023-01-25 10:09:35,2023-01-25 00:00:00
6,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,16366146,0x25fdda95197f804316f5ba8dbb66ac173ee19981,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.252766e+18,2.252766,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,2023-01-09 02:06:59,2023-01-09 00:00:00
7,0x3cf3a5658c3ef1a247ec241370a3abfeb673251eafec...,16176122,0x791d1ec51d931186c1d4b80e753b7155dd98f741,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.000000e+16,0.020000,0x3cf3a5658c3ef1a247ec241370a3abfeb673251eafec...,2022-12-13 13:37:23,2022-12-13 00:00:00
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,2.500004e+19,25.000039,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,2023-02-15 15:27:47,2023-02-15
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,True,gOHM,18,2855.939479,6.439304e+19,64.393035,0x85b689883076aa92af0a2356f8e4084610578bb48590...,2023-02-15 14:49:59,2023-02-15


In [258]:
path =file+'/siloFinance_withdrawals.csv'
withdrawalsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_withdrawals_piped.csv'
withdrawalsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', len(withdrawalsDbTimed),' records')

/Users/ianbrillembourg/Dropbox/Documents/Edgecaser LLC/Projects/Olympus/Econohmetrics/Lending/results/Collateral/siloFinance_withdrawals_piped.csv file saved, 16  records


In [259]:
dailySiloCollateral = withdrawalsDbTimed.groupby('calDate')['naturalAmount'].sum()
dailySiloCollateral.head(100)

calDate
2022-12-13 00:00:00     0.020000
2023-01-09 00:00:00     2.252766
2023-01-25 00:00:00    20.000000
2023-02-07 00:00:00     0.194798
2023-02-09 00:00:00    19.607121
2023-02-14 00:00:00    36.000000
2023-02-15 00:00:00    89.393074
2022-12-13              0.020000
2023-01-09              2.252766
2023-01-25             20.000000
2023-02-07              0.194798
2023-02-09             19.607121
2023-02-14             36.000000
2023-02-15             89.393074
Name: naturalAmount, dtype: float64

In [260]:
collateral = sum(withdrawalsDbTimed['naturalAmount'])
print('There are',round(collateral,2),  'gOHM available to borrow')

There are 334.94 gOHM available to borrow
